# Knapsack Optimization with KOPPU

This notebook demonstrates solving the Knapsack problem, a constrained optimization problem. We aim to maximize the value of items in a knapsack without exceeding its capacity.

In [ ]:
import pykoppu as pk

# 1. Define Items and Capacity
items = [
    {'name': 'Item 1', 'weight': 2, 'value': 10},
    {'name': 'Item 2', 'weight': 3, 'value': 15},
    {'name': 'Item 3', 'weight': 5, 'value': 20},
    {'name': 'Item 4', 'weight': 7, 'value': 25},
    {'name': 'Item 5', 'weight': 1, 'value': 5},
    {'name': 'Item 6', 'weight': 4, 'value': 12},
    {'name': 'Item 7', 'weight': 1, 'value': 3}
]
capacity = 10
penalty = 10.0 # Penalty strength for constraint violation

# 2. Create Knapsack Problem
print("Mapping to OPU...")
problem = pk.problems.logistics.Knapsack(items, capacity, penalty)

# 3. Solve
print("Solving...")
process = pk.oos.Process(problem)
result = process.run()

# 4. Display Results
print(f"Solution Valid? {result.metrics['valid']}")
print(f"Total Value: {result.metrics['total_value']}")
print(f"Total Weight: {result.metrics['total_weight']} / {capacity}")

# Show selected items
print("\nSelected Items:")
for i, item in enumerate(items):
    if result.solution[i] > 0.5:
        print(f"- {item['name']} (W: {item['weight']}, V: {item['value']})")

# 5. Visualize OPU Dynamics
print("\nVisualizing OPU Dynamics...")
result.plot()